In [19]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import pandas as pd

In [4]:
def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns true if the response seems to be HTML, false otherwise
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)

In [46]:
raw_html = simple_get('http://www.espn.com/nfl/statistics/player/_/stat/passing/sort/passingYards/year/2017/seasontype/2')
html = BeautifulSoup(raw_html, 'html.parser')
# print(html.prettify())
table = html.find_all('table')[0]
new_table = pd.DataFrame(columns=range(0,14), index=range(0,44))

row_marker = -1
for row in table.find_all('tr'):
    row_marker += 1
    row_marker
    column_marker = 0
    columns = row.find_all('td')
    for column in columns:
        new_table.iat[row_marker,column_marker] = column.get_text()
        column_marker += 1

# pd.DataFrame(columns=range(0,2), index = [0])

new_table

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,RK,PLAYER,TEAM,COMP,ATT,PCT,YDS,YDS/A,LONG,TD,INT,SACK,RATE,YDS/G
1,1,"Tom Brady, QB",NE,385,581,66.3,"4,577",7.88,64,32,8,35,102.8,286
2,2,"Philip Rivers, QB",LAC,360,575,62.6,"4,515",7.85,75,28,10,18,96.0,282
3,3,"Matthew Stafford, QB",DET,371,565,65.7,"4,446",7.87,71,29,10,47,99.3,278
4,4,"Drew Brees, QB",NO,386,536,72.0,"4,334",8.09,54,23,8,20,103.9,271
5,5,"Ben Roethlisberger, QB",PIT,360,561,64.2,"4,251",7.58,97,28,14,21,93.4,283
6,6,"Matt Ryan, QB",ATL,342,529,64.7,"4,095",7.74,88,20,12,24,91.4,256
7,7,"Kirk Cousins, QB",WSH,347,540,64.3,"4,093",7.58,74,27,13,41,93.9,256
8,8,"Alex Smith, QB",KC,341,505,67.5,"4,042",8.00,79,26,5,35,104.7,269
9,9,"Russell Wilson, QB",SEA,339,553,61.3,"3,983",7.20,74,34,11,43,95.4,249


In [43]:
column_marker

14